<a href="https://colab.research.google.com/github/HannesKock/RaceTeam2_CHP/blob/main/Race_Team_Project_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Race Team Project Notebook**
Names:

Matr.:

Please write down the Name of the Group member you worked on each section of code. This is necessary for grading by Studienbüro.

## **Analytics for Race 1**

- For each race day built one section with the code that you built for this race
- If you keep the same code during the next races copy the code over to the next section
- Describe why you built your analytics the way you do and interprete after each race what went well and what you want to improve on
- During motivation and interpretation try to cite online sources that you read to make better decisions (e.g., scientific articles on machine learning, blog posts, github pages)

In [ ]:
# Team members working on this code: Names..

In [ ]:
# Team members working on this code: Names..

In [ ]:
#..

In [ ]:
# Daten einlesen & Überblick verschaffen
import pandas as pd

df = pd.read_csv("simulator_data.csv")
print(df.head())
print(df.columns)
print(df.describe())
print(df.info())


In [ ]:
# Scatterplots (Parameter vs. Rundenzeit)
# Ziel: Visuelle Hinweise auf lineare / nichtlineare Abhängigkeiten

import matplotlib.pyplot as plt
import seaborn as sns

param_cols = df.columns.drop("Lap Time")

for col in param_cols:
    plt.figure()
    sns.scatterplot(x=df[col], y=df["Lap Time"], alpha=0.3)
    plt.title(f"{col} vs. Lap Time")
    plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Durchschnittsgeschwindigkeit berechnen (falls noch nicht vorhanden)
if "Avg. Speed" not in df.columns:
    df["Avg. Speed"] = df["Lap Distance"] / (df["Lap Time"] / 3600)

# Alle Spalten außer "Lap Time" und "Avg. Speed" verwenden
param_cols = df.columns.drop(["Lap Time", "Avg. Speed"])

# Scatterplots erzeugen
for col in param_cols:
    plt.figure()
    sns.scatterplot(x=df[col], y=df["Avg. Speed"], alpha=0.3)
    plt.title(f"{col} vs. Avg. Speed")
    plt.xlabel(col)
    plt.ylabel("Avg. Speed (km/h)")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
!pip install optuna
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import optuna

# === 1. Daten einlesen ===
df = pd.read_csv("simulator_data.csv")

# Ziel- und Feature-Spalten
target_col = "Lap Time"
feature_cols = df.columns.drop(target_col)

# === 2. Modell trainieren ===
X = df[feature_cols]
y = df[target_col]

# Optional: Skalieren
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Training/Test-Split (z. B. für Validierung)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# === 3. Bayesian Optimization Setup (Optuna) ===

# Gegeben: Umgebungsbedingungen
fixed_conditions = {
    'Lap Distance': 3.7,
    'Cornering': 6,
    'Inclines': 20,
    'Camber': 44,
    'Grip': 1,
    'Wind (Avg. Speed)': 97,
    'Temperature': 29,
    'Humidity': 23,
    'Air Density': 70,
    'Air Pressure': 98,
    'Wind (Gusts)': 49,
    'Altitude': 31,
    'Roughness': 49,
    'Width': 29
}

def objective(trial):
    # Optimierbare Fahrzeugparameter
    params = {
        'Rear Wing': trial.suggest_float('Rear Wing', 0.0, 500),
        'Engine': trial.suggest_float('Engine', 0.0, 500),
        'Front Wing': trial.suggest_float('Front Wing', 0.0, 500),
        'Brake Balance': trial.suggest_float('Brake Balance', 0.0, 500),
        'Differential': trial.suggest_float('Differential', 0.0, 500),
        'Suspension': trial.suggest_float('Suspension', 0.0, 500),
    }

    # Kombinieren mit festen Bedingungen
    full_input = {**fixed_conditions, **params}
    X_input = pd.DataFrame([full_input])
    X_input_scaled = scaler.transform(X_input)

    # Vorhersage durch Modell
    lap_time = model.predict(X_input_scaled)[0]
    return lap_time

# Optuna-Studie starten
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Ergebnisse
print("Beste Parameterkombination:")
for key, value in study.best_params.items():
    print(f"  {key}: {value:.4f}")
print(f"Erwartete Rundenzeit: {study.best_value:.4f} Sekunden")

In [ ]:
from sklearn.inspection import PartialDependenceDisplay

# Modell (z. B. Random Forest muss vorher trainiert sein)
features = ['Rear Wing', 'Engine', 'Front Wing', 'Brake Balance', 'Differential', 'Suspension']
PartialDependenceDisplay.from_estimator(model, X, features, target='Lap Time')
plt.show()

In [ ]:
import statsmodels.api as sm

X = df[['Rear Wing', 'Engine', 'Front Wing', 'Brake Balance', 'Differential', 'Suspension']]
X = sm.add_constant(X)
y = df["Lap Time"]

model = sm.OLS(y, X).fit()
print(model.summary())

In [ ]:
import shap
import numpy  as np
import xgboost as xgb
from sklearn.model_selection import train_test_split

# 1. Falls noch nicht geschehen: Durchschnittsgeschwindigkeit berechnen
if "Avg. Speed" not in df.columns:
    df["Avg. Speed"] = df["Lap Distance"] / (df["Lap Time"] / 3600)

# 2. Features & Ziel definieren
feature_cols = [
    'Lap Distance', 'Cornering', 'Inclines', 'Camber', 'Grip',
    'Wind (Avg. Speed)', 'Temperature', 'Humidity', 'Air Density',
    'Air Pressure', 'Wind (Gusts)', 'Altitude', 'Roughness', 'Width',
    'Rear Wing', 'Engine', 'Front Wing', 'Brake Balance', 'Differential',
    'Suspension'
]

X = df[feature_cols]
y = df["Avg. Speed"]

# 3. Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.00000001)

# 4. Modell trainieren (XGBoost)
model = xgb.XGBRegressor(n_estimators=100, max_depth=5, random_state=42)
model.fit(X_train, y_train)
y_hat = model.predict(X_test)
error = np.mean(np.abs(y_test - y_hat))
print(error)

# # 5. SHAP-Werte berechnen
# explainer = shap.Explainer(model)
# shap_values = explainer(X_test)

# # 6. SHAP Summary Plot
# shap.summary_plot(shap_values, X_test)


In [ ]:
import optuna
import numpy as np
import pandas as pd

# You can get this from df.mean().values or pick a sample row
base_input = df[feature_cols].mean().values

for fc in fixed_conditions:
    base_input[feature_cols.index(fc)] = fixed_conditions[fc]

# Indices of the features we want to optimize
optim_features = ['Rear Wing', 'Engine', 'Front Wing', 'Brake Balance', 'Differential', 'Suspension']
optim_indices = [feature_cols.index(f) for f in optim_features]

# Define bounds from your dataset (here we use min/max)
feature_bounds = {
    'Rear Wing': (1, 500),
    'Engine': (1, 500),
    'Front Wing': (1, 500),
    'Brake Balance': (1, 500),
    'Differential': (1, 500),
    'Suspension': (1, 500),
}

# Objective function for Optuna
def objective(trial):
    x = base_input.copy()

    for f in optim_features:
        val = trial.suggest_int(f, int(feature_bounds[f][0]), int(feature_bounds[f][1]))
        x[feature_cols.index(f)] = val

    pred = model.predict(np.array([x]))[0]
    return pred  # Optuna will maximize if we tell it to

# Run Optuna study
optuna.logging.disable_default_handler()
study = optuna.create_study(direction='maximize')  # use 'minimize' if lower lap time is better
study.optimize(objective, n_trials=255)


# Show results
print("Best params:")
for k, v in study.best_params.items():
    print(f"{k}: {v}")

print(f"Max predicted avg speed: {study.best_value}")


In [ ]:
import optuna
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split

# 1. Falls noch nicht geschehen: Durchschnittsgeschwindigkeit berechnen
if "Avg. Speed" not in df.columns:
    df["Avg. Speed"] = df["Lap Distance"] / (df["Lap Time"] / 3600)

# 2. Features & Ziel definieren
feature_cols = [
    'Lap Distance', 'Cornering', 'Inclines', 'Camber', 'Grip',
    'Wind (Avg. Speed)', 'Temperature', 'Humidity', 'Air Density',
    'Air Pressure', 'Wind (Gusts)', 'Altitude', 'Roughness', 'Width',
    'Rear Wing', 'Engine', 'Front Wing', 'Brake Balance', 'Differential',
    'Suspension'
]

X = df[feature_cols]
y = df["Avg. Speed"]


# 4. XGBoost-Modell trainieren (Basis)
model = xgb.XGBRegressor(n_estimators=100, max_depth=5, random_state=42)
model.fit(X_train, y_train)

# 5. Zielfunktion für die Optimierung definieren
def objective(trial):
    # Parameter, die optimiert werden sollen
    params = {
        'Rear Wing': trial.suggest_int('Rear Wing', 0, 500),
        'Engine': trial.suggest_int('Engine', 0, 500),
        'Front Wing': trial.suggest_int('Front Wing', 0, 500),
        'Brake Balance': trial.suggest_int('Brake Balance', 0, 500),
        'Differential': trial.suggest_int('Differential', 0, 500),
        'Suspension': trial.suggest_int('Suspension', 0, 500)
    }

    # Testdaten mit den aktuellen Parameterwerten
    test_data = fixed_conditions.copy()
    test_data.update(params)

    # Das Modell verwenden, um Vorhersage der Durchschnittsgeschwindigkeit für diese Parameter zu treffen
    test_df = pd.DataFrame([test_data])
    X_test_params = test_df[feature_cols]

    # Vorhersage machen
    predicted_speed = model.predict(X_test_params)[0]

    # Das Ziel ist es, die Geschwindigkeit zu maximieren
    return -predicted_speed  # Negieren, da Optuna minimiert

# 6. Optuna-Optimierung starten
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Beste Parameterkombination
best_params = study.best_params
print("Beste Parameterkombination:", best_params)

# Vorhersage mit den besten Parametern
best_test_data = fixed_conditions.copy()
best_test_data.update(best_params)
best_test_df = pd.DataFrame([best_test_data])
X_best_test = best_test_df[feature_cols]

# Vorhersage der besten Durchschnittsgeschwindigkeit
best_predicted_speed = model.predict(X_best_test)[0]
print("Vorhergesagte beste Durchschnittsgeschwindigkeit:", best_predicted_speed)


## **Analytics for Race 2**

In [ ]:
# Team members working on this code: Names..

## **Analytics for Race 3**

In [ ]:
# Team members working on this code: Names..

## **Analytics for Race 4**

In [ ]:
# Team members working on this code: Names..

## **Analytics for Race 5**

In [ ]:
# Team members working on this code: Names..

## **Debrief Race Calendar before Final Race**

*Write a longer text (200-500 words) reflecting on what were the main ideas you started the seminar with, how you improved your models to achieve better performance and what strategy and analytics you want to use for your final race during seminar day*

## **Analytics for Final Race**

In [ ]:
# Team members working on this code: Names..

## **References**
- Cite all references you need according to chair guidelines

Liu, Xuan; Shi, Savannah Wei; Teixeira, Thales; Wedel, Michel (2018): Video Content Marketing: The Making of Clips, Journal of Marketing, Vol. 82, 86-101.